# Metapopulation Agent-Based SEIR Model

## Framework 
* Data collection and wrangling in Python 
* Modelling in Julia 

## Modules

In [1]:
# Data Management 
using DataFrames, DataFramesMeta, DrWatson, Queryverse
# Statistics
using Random, StatsBase, Distributions
# Graphs 
using LightGraphs, SimpleWeightedGraphs, GraphIO, GraphPlot
# Modelling
using Agents
# Numerical Computation 
using LinearAlgebra
# Data Visualization
using Plots, AgentsPlots, PlotThemes
# Python Interface
using PyCall 
nx = pyimport("networkx");

## Data 

### Administrative

In [2]:
### load("/Users/pietromonticone/github/DigitalEpidemiologyProject/Data/CSV/2020/??/Provinces.csv");

### Population 

In [3]:
population_data = DataFrame(load("/Users/pietromonticone/github/DigitalEpidemiologyProject/Data/CSV/2020/ProvincialPopulation.csv"));
age_population_data = DataFrame(load("/Users/pietromonticone/github/DigitalEpidemiologyProject/Data/CSV/2020/AgeStratifiedProvincialPopulation.csv"));

### Contact 

In [4]:
# Load contact data
contact_data = DataFrame(load("/Users/pietromonticone/github/DigitalEpidemiologyProject/Data/CSV/2020/ContactEdgeList.csv"))
# Build contact graph 
contact_graph = SimpleWeightedDiGraph(contact_data[1].+1, contact_data[2].+1, contact_data[3]);

{16, 256} directed simple Int64 graph with Float64 weights

### Mobility

In [5]:
# Load mobility data
mobility_data = DataFrame(load("/Users/pietromonticone/github/DigitalEpidemiologyProject/Data/CSV/2020/MobilityFlow.csv"))
# Build mobility graph 
mobility_graph = SimpleWeightedDiGraph(mobility_data[1], mobility_data[2], mobility_data[3])

# Remove the 104th node: provincial code doesn't exist
rem_vertex!(mobility_graph, 104)
# Remove the 105th node: provincial code doesn't exist
rem_vertex!(mobility_graph, 104)
# Remove the 106th node: provincial code doesn't exist
rem_vertex!(mobility_graph, 104)
# Remove the 107th node: provincial code doesn't exist
rem_vertex!(mobility_graph, 104);

## Agent Type 

In [6]:
# Voter Definition
mutable struct Patient <: AbstractAgent
    id::Int          # identity code ∈ ℕ               
    pos::Int         # location node ∈ ℕ 
    age_group::Int  # age group ∈ [1,16] ⊂ ℕ
    home::Int        # household node ∈ ℕ
    #work::Int       # workplace node ∈ ℕ
    status::Symbol   # ∈ {S,E,I,R}
end;

## Behaviors

In [7]:
function migrate!(agent, model)
    source = agent.pos
    x = [outneighbor for outneighbor in LightGraphs.weights(model.space.graph)[source,:]]
    #outneighbors = node_neighbors(agent, model; neighbor_type=:out)
    d = DiscreteNonParametric(1:(model.M), x./sum(x))
    target = rand(d)
    if target ≠ source
        move_agent!(agent, target, model)
    end
end

function agent_step!(agent, model)
    migrate!(agent, model)
    #contact!(agent, model)
    #transmit!(agent, model)
    #update!(agent, model)
    #recover_or_die!(agent, model)
end;

## Parameters

In [14]:
# Rescaled Populations
provincial_pops = round.(Int, population_data["population"]./000) # array of subpopulation / provincial sizes
age_provincial_pops = round.(Int, age_population_data./7000)
N = sum(provincial_pops)      # total number of agents
M = length(provincial_pops)   # number of subpopulations / provinces 

# Transition rates 
α = 0.2  # S -> E 
β = 0.2  # E -> I
γ = 0.2  # I -> R 

# Initial conditio

LoadError: InexactError: trunc(Int64, Inf)

## Model

In [9]:
# Model initialization 
function initialize_model(provincial_pops::Array{Int,1}, age_provincial_pops::DataFrame,
                          contact_graph::SimpleWeightedDiGraph{Int64,Float64},
                          mobility_graph::SimpleWeightedDiGraph{Int64,Float64}, 
                          α::Real, β::Real, γ::Real,
                          I0)
    # Set the total number of agents
    N = sum(provincial_pops)
    # Set the number of subpopulations
    M = length(provincial_pops)
    # Set the number of age groups 
    K = length(age_provincial_pops[1]) # 16 age classes 
    # Initialize id 
    id = 0    
    
    # Define the dictionary of model properties
    properties = @dict(contact_graph, 
                       provincial_pops, age_provincial_pops,
                       α, β, γ, I0,
                       N, M)
    
    # Instantiate graph ambient space
    space = GraphSpace(mobility_graph) 
    # Instatiate model
    model = ABM(Patient, space; properties = properties)
        
    # Populate the model
    for province ∈ 1:M
        for age ∈ 1:K 
            for n ∈ 1:age_provincial_pops[province][age]
                id+=1
                home=province
                pos=home
                age_group=age
                status=:S
                add_agent!(pos, model, age_group, home, status)
            end
        end
    end
    
    return model
end;

## Simulations

In [10]:
model = initialize_model(provincial_pops, age_provincial_pops, contact_graph, mobility_graph,
                         α, β, γ, I0)

simulation_data = @time run!(model, agent_step!, 100);

sort!(DataFrame(allagents(model)), :id, rev = false)

LoadError: UndefVarError: provincial_pops not defined

## Visualizations 

In [11]:
#plotargs = (node_size = 0.2, method = :spring, linealpha = 0.4)
#plotabm(model; plotargs...)

In [12]:
#delta_pop(x) = cgrad(:inferno)[count(a.pos == a.home for a in x) / length(x)]

In [13]:
model = initialize_model(provinces, contact_graph, mobility_graph,
                         α, β, γ, I0)

g = model.space.graph
weight_matrix = LightGraphs.weights(g)
for i in 1:nv(g)
    weight_matrix[i,i] = 0
end

edgewidthsdict = Dict()
for node in 1:nv(g)
    nbs = neighbors(g, node)
    for nb in nbs
        edgewidthsdict[(node, nb)] = weight_matrix[node,nb]/sum([outneighbor for outneighbor in weight_matrix[node,:]])
    end
end

edgewidthsf(s, d, w) = edgewidthsdict[(s, d)] * 10
plotargs = (node_size = 0.2, method = :spring, linealpha = 0.4)

plotargs = merge(plotargs, (edgewidth = edgewidthsf,))

#plotabm(model; ac = delta_pop, plotargs...)

LoadError: UndefVarError: provinces not defined

![](animation.gif)